# Faiss 使用

Faiss 用于支持搜索任何大小的向量，也支持 evaluation 和模型调参，大部分算法支持使用 GPU。

In [ ]:
try:
    import faiss
    print('Faiss installed.')
except:
    print('Faiss not installed. Maybe only support linux.')
import numpy as np

## 1. Demo 简单搜索

### 0x1 构建数据和索引

首先构建数据

In [ ]:
# dimension
d = 64
# database size
nb = 100000
# quries
nq = 10000
np.random.seed(1234)
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

创建索引，并且为索引添加数据。

Faiss 用于包装数据库向量，可以提前进行预处理让搜索更高效。

有很多索引类型，最简单的 L2 距离索引为 `IndexFlatL2`

构建和训练索引分别对应两个操作：`add` 和 `search`

大部分分索引需要训练 `is_trained` 来学习得到数据的分布情况，这个索引不需要，所以在此忽略。

In [ ]:
index = faiss.IndexFlatL2(d)
print(index.is_trained)
index.add(xb)
print('包含的向量数量', index.ntotal)

### 0x2 搜索

此处搜索使用最简单的 KNN 来进行搜索操作。

In [ ]:
k = 4
# 正确性检查
distance, idx = index.search(xb[:5], k)
idx, distance

`search` 返回的结果是 distance, idx，分别表示与目标搜索向量的距离和对应的索引

In [ ]:
import time
s = time.time()
D, I = index.search(xq, k)
print(f'cost {time.time() - s:.2f}s')

In [ ]:
I[:5], I[-5:]

## 2. 快速搜索

搜索速度太慢了，如何更快搜索？将每块数据进行分区处理，使用一个代表性的向量来表示这个数据分片，如果 query 和这个代表性向量比较相近，则只需要对比这一个分区内的所有向量，从而达到较少搜索次数加快搜索速度的目的。

`IndexIVFFlat` [倒排索引](https://www.elastic.co/guide/cn/elasticsearch/guide/current/inverted-index.html)就实现了这个功能，`Flat` 表示数据没有经过编码压缩的意思。

这类索引需要训练的过程，并且需要其他的索引(quantizer)用于将向量分配到 Voronoi cell 中，每个 cell 都是用一个中心向量来用于寻找最近邻，通常这个索引是 `IndexFlatL2`。

有两个参数可以进行搜索策略调整：
* `nlist`: 用来指定 cell 的数量，相当于聚类中心的数量
* `nprobe`: 用来指定在执行搜索的时候，每次需要查询的聚类中心数量

In [ ]:
nlist = 100
k = 4
iv_index = faiss.IndexIVFFlat(index, d, nlist)
print(iv_index.is_trained)
iv_index.train(xb)
iv_index.is_trained

In [ ]:
iv_index.add(xb)
s = time.time()
iv_index.nprobe = 1
D, I = iv_index.search(xq, k)
print(f'cost {time.time() - s:.2f}s')
D, I

In [ ]:
s = time.time()
iv_index.nprobe = 10
D, I = iv_index.search(xq, k)
print(f'cost {time.time() - s:.2f}s')
D, I

In [ ]:
s = time.time()
iv_index.nprobe = 100
D, I = iv_index.search(xq, k)
print(f'cost {time.time() - s:.2f}s')
D, I

可以观察到，当 `nprobe` 数量越高的时候，越接近于暴力搜索，因此设置 `nprobe` 可以在执行速度和精度之间进行权衡。

## 3. 节约内存搜索

在一些情况下可能保存的数据较大，会导致内存占用过多，因此 faiss 提供了一种有损压缩来进行减小空间存储。

这种计数基于 [Product Quantizer](https://hal.archives-ouvertes.fr/file/index/docid/514462/filename/paper_hal.pdf) 因此采取压缩的类多以 `xxxPQ` 结尾，在这种情况下各种数据可能会有精度损失。


In [ ]:
m = 8  # 子采样器的数量
index = faiss.IndexFlatL2(d)
pq_index = faiss.IndexIVFPQ(index, d, nlist, m, 8) # 末尾的 8 表示使用 8 个子采样器将每个向量压缩成 8 bit
pq_index.train(xb)
pq_index.add(xb)
D, I = pq_index.search(xb[:5], k) # sanity check
print(I)
print(D)
pq_index.nprobe = 10              # make comparable with experiment above
D, I = pq_index.search(xq, k)     # search
print(I[-5:])

## 4. 工厂方法

快速创建索引

In [ ]:
index = faiss.index_factory(d,"PCA32,IVF100,PQ8")

该字符串的含义为：使用PCA算法将向量降维到32维, 划分成100个nprobe (搜索空间), 通过PQ算法将每个向量压缩成8bit。

In [ ]:
index = faiss.index_factory(d,"IVF100,FLAT")

该字符串的含义为：划分成100个nprobe (搜索空间), 不压缩。

## 5. 使用 GPU

使用单个 GPU，每个索引需要至少 `256MB` 的空间。

In [ ]:
res = faiss.StandardGpuResources()  # use a single GPU

In [ ]:
# build a flat (CPU) index
index_flat = faiss.IndexFlatL2(1)
# make it into a gpu index
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)

通过函数 `index_cpu_to_all_gpus` 使用多个 GPU

In [ ]:
ngpus = faiss.get_num_gpus()

print("number of GPUs:", ngpus)

cpu_index = faiss.IndexFlatL2(d)

gpu_index = faiss.index_cpu_to_all_gpus(  # build the index
    cpu_index
)

gpu_index.add(xb)              # add vectors to the index
print(gpu_index.ntotal)

k = 4                          # we want to see 4 nearest neighbors
D, I = gpu_index.search(xq, k) # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])      

## 6. 构建余弦相似度

faiss 中不支持之间构建余弦相似度，在[此](https://github.com/facebookresearch/faiss/wiki/MetricType-and-distances)提到构建余弦相似度的步骤：

1. 创建索引 `IndexFlatIP`
2. 在添加到索引之前使用进行 L2 归一化（`faiss.normalize_L2` in python）
3. 搜索

## 7. 保存和读取索引

In [ ]:
faiss.write_index(index, 'a.index')
index = faiss.read_index('a.index')

## 8. 向量与 ID 对应

In [ ]:
import numpy as np
import faiss
# dimension
d = 5
# database size
nb = 100
# quries
nq = 5
np.random.seed(1234)
xb = np.random.random((nb, d)).astype('float32')
xq = np.random.random((nq, d)).astype('float32')
xb
ids = np.arange(nb, nb*2).astype(np.int64)
nlist = 10
quantizer = faiss.IndexFlatIP(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_INNER_PRODUCT)
index.train(xb)
index.add_with_ids(xb, ids)
index.search(xq, 1000)